In [154]:
import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [155]:
env = gym.make('CartPole-v0')

In [156]:
class NetW(nn.Module):
    def __init__(self):
        print('net created')
        super().__init__()
        
        self.commonlayer = nn.Linear(4, 128)
        
        self.actorhead = nn.Linear(128, 2)
        
        self.critichead = nn.Linear(128, 1)
        
        self.prob_val = []
        self.rewards = []
    
    def forward(self, state):
       # print('forward')        
        state_ = torch.from_numpy(state).float()
        x = self.commonlayer(state)
        prob = F.softmax(self.actorhead(x), dim=-1)
        val  = self.critichead(x)
        print(state)
        print(prob)
        return prob, val

In [157]:
policy = NetW()

net created


In [158]:
optimizer = optim.Adam(policy.parameters(), lr=3e-2)

In [159]:
def getAction(state):
    print('action....')
    print(state)
    p, v = policy(state)
    m = Categorical(p)
    action = m.sample()
    policy.prob_val.append((m.log_prob(action), v))
    return action.item()

In [160]:
def getReturns():
    R = 0
    returns = [] # list to save the true values

    # calculate the true value using rewards returned from the environment
    # 
    for r in policy.rewards[::-1]:
        # calculate the discounted value
        R = r + gamma * R
        returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std())
    return returns

In [167]:
def select_action(state):
    print('----1')
    state = torch.from_numpy(state).float()
    #to_device(state, device)
    print('----2', state)
    
    probs, state_value = policy(state)
    print('----3')
    # create a categorical distribution over the list of probabilities of actions
    m = Categorical(probs)

    # and sample an action using the distribution
    action = m.sample()

    # save to action buffer
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))

    # the action to take (left or right)
    return action.item()

In [168]:
def learn():
    R = 0
    R = getReturns()
    losses = []
    value_losses= []
   # print('learn')
    for p, r in zip(policy.prob_val, R):
        prob, val = p
        adv = r - val.item()
        loss = -prob * adv
        losses.append(loss)
        value_losses.append(F.smooth_l1_loss(val, torch.tensor([r])))
    optimizer.zero_grad()

    # sum up all the values of policy_losses and value_losses
    loss = torch.stack(losses).sum() + torch.stack(value_losses).sum()

    # perform backprop
    loss.backward()
    optimizer.step()

    # reset rewards and action buffer
    del policy.rewards[:]
    del policy.prob_val[:]       
            

In [169]:
done = False
state = env.reset()
gamma = 0.99
for y in range(200):
    state = env.reset()
    rewards = []
    total_rewards  = 0
    for x in range(1000):
        action = select_action(state)
        action = getAction()
        state_, reward, done, info = env.step(action)
        policy.rewards.append(reward)
        rewards.append(reward)
        total_rewards = total_rewards + reward
        learn()
        if done == True:
            print(total_rewards)
            break

----1
----2 tensor([ 0.0154,  0.0369, -0.0086,  0.0442])


TypeError: expected np.ndarray (got Tensor)

In [130]:
*env.step(1)

SyntaxError: can't use starred expression here (<ipython-input-130-fa5fba9b0afc>, line 4)

In [31]:
env.reset()

array([ 0.00492423, -0.0348803 ,  0.03119667, -0.03710701])

In [32]:
env.step(1)

(array([ 0.00422662,  0.15978071,  0.03045453, -0.31978613]), 1.0, False, {})